[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/trainer/blob/main/trainer.ipynb)

In [ ]:
%cd /content

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
%env TF_CPP_MIN_LOG_LEVEL=1

MountDrive = False #@param ["False", "True"] {type:"raw"}

DownloadModel = True #@param ["False", "True"] {type:"raw"}
model_raw_url = "https://huggingface.co/uf/cyberrealistic_v3.2/raw/main" #@param {type:"string"}
model_resolve_url = "https://huggingface.co/uf/cyberrealistic_v3.2/resolve/main" #@param {type:"string"}
output_dir = "/content/model" #@param {type:"string"}

DownloadTestModel = False #@param ["False", "True"] {type:"raw"}
test_model_raw_url = "https://huggingface.co/camenduru/testA/raw/main/output_dir_1250" #@param {type:"string"}
test_model_resolve_url = "https://huggingface.co/camenduru/testA/resolve/main/output_dir_1250" #@param {type:"string"}
test_output_dir = "/content/trainer/diffusers/dreambooth/output_dir" #@param {type:"string"}

if MountDrive:
  from google.colab import drive
  drive.mount('/content/drive')

!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 diffusers==0.19.0 datasets==2.14.0 gradio==3.38.0 wandb==0.15.7 transformers==4.26.0 accelerate==0.16.0 -U

!git clone https://github.com/camenduru/trainer
%cd /content/trainer

diffusers_version = "v0.19.0"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/scripts/convert_diffusers_to_original_stable_diffusion.py -d /content/trainer/diffusers/dreambooth -o convert_diffusers_to_original_stable_diffusion.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/dreambooth/train_dreambooth.py -d /content/trainer/diffusers/dreambooth -o train_dreambooth.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/dreambooth/train_dreambooth_lora.py -d /content/trainer/diffusers/lora -o train_dreambooth_lora.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/text_to_image/train_text_to_image.py -d /content/trainer/diffusers/text_to_image -o train_text_to_image.py
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M  https://raw.githubusercontent.com/huggingface/diffusers/{diffusers_version}/examples/textual_inversion/textual_inversion.py -d /content/trainer/diffusers/textual_inversion -o textual_inversion.py

if DownloadModel:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/model_index.json -d {output_dir} -o model_index.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_resolve_url}/vae/diffusion_pytorch_model.bin -d {output_dir} -o vae/diffusion_pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/vae/config.json -d {output_dir} -o vae/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_resolve_url}/unet/diffusion_pytorch_model.bin -d {output_dir} -o unet/diffusion_pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/unet/config.json -d {output_dir} -o unet/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/tokenizer/vocab.json -d {output_dir} -o tokenizer/vocab.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/tokenizer/tokenizer_config.json -d {output_dir} -o tokenizer/tokenizer_config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/tokenizer/special_tokens_map.json -d {output_dir} -o tokenizer/special_tokens_map.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/tokenizer/merges.txt -d {output_dir} -o tokenizer/merges.txt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_resolve_url}/text_encoder/pytorch_model.bin -d {output_dir} -o text_encoder/pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/text_encoder/config.json -d {output_dir} -o text_encoder/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/scheduler/scheduler_config.json -d {output_dir} -o scheduler/scheduler_config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_resolve_url}/safety_checker/pytorch_model.bin -d {output_dir} -o safety_checker/pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/safety_checker/config.json -d {output_dir} -o safety_checker/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_raw_url}/feature_extractor/preprocessor_config.json -d {output_dir} -o feature_extractor/preprocessor_config.json

if DownloadTestModel:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/model_index.json -d {test_output_dir} -o model_index.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_resolve_url}/vae/diffusion_pytorch_model.bin -d {test_output_dir} -o vae/diffusion_pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/vae/config.json -d {test_output_dir} -o vae/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_resolve_url}/unet/diffusion_pytorch_model.bin -d {test_output_dir} -o unet/diffusion_pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/unet/config.json -d {test_output_dir} -o unet/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/tokenizer/vocab.json -d {test_output_dir} -o tokenizer/vocab.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/tokenizer/tokenizer_config.json -d {test_output_dir} -o tokenizer/tokenizer_config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/tokenizer/special_tokens_map.json -d {test_output_dir} -o tokenizer/special_tokens_map.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/tokenizer/merges.txt -d {test_output_dir} -o tokenizer/merges.txt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_resolve_url}/text_encoder/pytorch_model.bin -d {test_output_dir} -o text_encoder/pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/text_encoder/config.json -d {test_output_dir} -o text_encoder/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/scheduler/scheduler_config.json -d {test_output_dir} -o scheduler/scheduler_config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_resolve_url}/safety_checker/pytorch_model.bin -d {test_output_dir} -o safety_checker/pytorch_model.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/safety_checker/config.json -d {test_output_dir} -o safety_checker/config.json
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {test_model_raw_url}/feature_extractor/preprocessor_config.json -d {test_output_dir} -o feature_extractor/preprocessor_config.json

%cd /content/trainer
!python trainer.py

In [ ]:
#@title Push to HF.co

import os
from huggingface_hub import create_repo, upload_folder

hf_token = 'HF_WRITE_TOKEN' #@param {type:"string"}
repo_id = 'username/reponame' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
model_path = '/content/train/model' #@param {type:"string"}
upload_folder(folder_path=f'{model_path}', path_in_repo='', repo_id=repo_id, commit_message=commit_message, token=hf_token)

In [ ]:
#@title Push to DagsHub.com

!pip -q install dagshub
from dagshub.upload import Repo, create_repo

repo_id = 'reponame' #@param {type:"string"}
org_name = 'orgname' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(f"{repo_id}", org_name=f"{org_name}")
repo = Repo(f"{org_name}", f"{repo_id}")
model_path = '/content/train/model' #@param {type:"string"}
repo.upload("/content/models", remote_path="data", commit_message=f"{commit_message}", force=True)